# 数据处理

In [ ]:
import pandas as pd 
import zipfile
import os 
import py7zr
import multiprocessing 
from tqdm import tqdm 
from concurrent.futures import ThreadPoolExecutor 

In [ ]:
# 假设压缩文件名为 data.zip
zip_file_path1 = 'data/basketball_reference.7z'
zip_file_path2 = 'data/basketball_reference_part2_finish.7z'

# 解压缩文件到指定目录
extract_folder = 'data/nba'
# 如果不存在
if not os.path.exists(extract_folder):
    with py7zr.SevenZipFile(zip_file_path1, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    with py7zr.SevenZipFile(zip_file_path2, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        
file_path_list = []
df_list = []
# 遍历解压后的文件夹，并读取每个 CSV 文件
for root, dirs, files in os.walk(extract_folder):
    for file in files:
        if file.endswith("basic_stats.csv"):
            file_path = os.path.join(root, file)
            file_path_list.append(file_path)
            print(file_path)
            # 使用 pandas 读取 CSV 文件
            df_basic = pd.read_csv(file_path).drop_duplicates()
            df_advanced = pd.read_csv(file_path.replace('basic_stats.csv','advanced_stats.csv')).drop_duplicates()
            if df_basic['PTS'].dtype=='object':
                df_basic = df_basic.drop(df_basic.index[-1])
                df_advanced = df_advanced.drop(df_advanced.index[-1])
            # 连接两个文件：63162
            df = pd.concat([df_basic, df_advanced], axis=1)
            df = df.loc[:,~df.columns.duplicated()]
            # 将文件名作为新列添加到 DataFrame
            df['file_name'] = file_path.split('/')[-1]
            df_list.append(df)

print(len(file_path_list),len(df_list))  

In [ ]:
# # 返回所有最大值的索引
# len_df = [len(df_list[i]) for i in range(len(df_list))]
# max_player = max(len_df)
# max_indices = [i for i, v in enumerate(len_df) if v == max_player]
# print(max_player, max_indices)
# print(df_list[max_indices[0]]['starters'])

In [ ]:
# print(file_path_list[0].split('/')[-1])
len_df = [len(df_list[i]) for i in range(len(df_list))]
max_player = max(len_df)
print('max_player:',max_player)
# 打印所有列名
# print(df_list[0].columns)

features = ['starters', 'team', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM',
       'file_name']
# 连接所有球员数据
new_features = []
for i in range(max_player*2):
    for feat in features:
        new_feat = feat + '_' + str(i)
        new_features.append(new_feat)
print(len(features), len(new_features))

result_features = ['score_ourside', 'score_opposite', 'result']
new_data = pd.DataFrame(columns = new_features + result_features)
from tqdm import tqdm
for i in tqdm(range(len(file_path_list))):
# for i in range(10):
    path = file_path_list[i]
    # print(path.split('/')[-1])
    df1 = df_list[i]
    filled_df1 = df1.reindex(range(max_player))
    #对手球队数据
    a = path.split('/')[-1].split('_')[3]
    b = path.split('/')[-1].split('_')[5]
    oppo_path = path.replace(a+'_vs_'+b, b+'_vs_'+a)
    # print(oppo_path,file_path_list.index(oppo_path))
    df2 = df_list[file_path_list.index(oppo_path)]
    filled_df2 = df2.reindex(range(max_player))
    # print(filled_df1["PTS"],filled_df2["PTS"])
    score_ourside = filled_df1["PTS"].astype(float).sum()
    score_opposite = filled_df2["PTS"].astype(float).sum()
    if score_ourside > score_opposite:
        result = 1
    else:
        result = 0
    
    feature_data1 = []
    feature_data2 = []
    for num in range(max_player):
        feature_data1 += filled_df1.loc[num, features].tolist() 
        feature_data2 += filled_df2.loc[num, features].tolist() 
        
    new_data.loc[i, new_features + result_features] = feature_data1 + feature_data2 + [score_ourside, score_opposite, result]

new_data.to_csv('./data/nba_mvp1.csv')

print(len(new_data))


In [ ]:
# # file_path = 'data/nba/basketball_reference_part2_finish/2023_12_27/2023_12_27_OKC_vs_NYK_basic_stats.csv'
# file_path = 'data/nba/basketball_reference/2000_01_26/2000_01_26_MIL_vs_SAC_basic_stats.csv'
# df_basic = pd.read_csv(file_path).drop_duplicates()
# print(df_basic['PTS'].dtype=='object')
# # print(df_basic['PTS'].sum())
# df_basic = df_basic.drop(df_basic.index[-1])
# print(df_basic['PTS'].astype(float).sum())

# 训练胜负模型

In [ ]:
import pandas as pd 
import zipfile
import os 
import py7zr
import multiprocessing 
from tqdm import tqdm 
import numpy as np
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt
# from plot_helper import dependence_plot_v12
import multiprocessing
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor 
import numpy as np
import pandas as pd
import lightgbm
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, classification_report
from lightgbm import log_evaluation, early_stopping
from sklearn.preprocessing import KBinsDiscretizer
plt.rcParams['axes.unicode_minus']=False

DRtg_bins = 3
ORtg_bins = 20
DRtg_Fuzzification = True
DRtg_Fuzzification = False
ORtg_Fuzzification = True
# ORtg_Fuzzification = False
def data_load(data):
    df = data
    # print(df.info())
    # print(df.columns)
    # 将时间转换为浮点数
    time_features = []
    for i in range(36):
        feature = 'MP' + '_' + str(i)
        time_features.append(feature)
    # print(time_features) 
    
    df = df[~df['BPM_0'].astype(str).str.contains(':')]
    # print(df['BPM_0'].astype(str).str.contains(':'))
    print('清洗后数据：',len(df))
    df[time_features] = df[time_features].fillna('0:0')
    df[time_features] = df[time_features].map(lambda x: '0:0' if ':' not in str(x) else x)
    df[time_features] =df[time_features].map(lambda x: float(x.replace(':', '.')))
    
    if  DRtg_Fuzzification:
        #  #缩小'ORtg','DRtg'影响
        norm_features = []
        for i in range(36):
            for feat in ['DRtg']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
        df = df.fillna(0)
        # # 转换为数值类型
        df[norm_features] = df[norm_features].apply(pd.to_numeric, errors='coerce')
        # 将负值变为0
        df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)
        # 将 norm_features 值聚合在一起
        norm_feature_values = df[norm_features].values.flatten().reshape(-1, 1)    
        print(norm_feature_values.shape)
        # 将数据划分为 10 个区间进行数值离散化处理
        kdb = KBinsDiscretizer(n_bins = DRtg_bins, encode = 'ordinal', strategy='quantile')
        kdb.fit(norm_feature_values)
        for norm_feature in norm_features:
            df[norm_feature] = kdb.transform(df[norm_feature].values.reshape(-1, 1))
    if  ORtg_Fuzzification:
        norm_features = []
        for i in range(36):
            for feat in ['ORtg']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
        # # 转换为数值类型
        df[norm_features] = df[norm_features].apply(pd.to_numeric, errors='coerce')
        # 将负值变为0
        df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)
        # 将 norm_features 值聚合在一起
        norm_feature_values = df[norm_features].values.flatten().reshape(-1, 1)    
        print(norm_feature_values.shape)
        # 将数据划分为 10 个区间进行数值离散化处理
        kdb = KBinsDiscretizer(n_bins = ORtg_bins, encode = 'ordinal', strategy='quantile')
        kdb.fit(norm_feature_values)
        for norm_feature in norm_features:
            df[norm_feature] = kdb.transform(df[norm_feature].values.reshape(-1, 1))
        # print(df[['DRtg_0','DRtg_35']])
        # print(df[['ORtg_0','ORtg_35']])
    print(df.info())
    print(df.columns)
    return df

def data_prerpocess(df, training = True):
    
    #V_0：全有
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_1去除+-，DRtg
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'BPM']
    #V_2去除MP,+-，DRtg
    # used_features = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','BPM']
    #v_3 +-,'BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','DRtg']
    # V_4去除PTS,+-,'ORtg','BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_5 去除+-,'ORtg','DRtg', 'BPM'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_6 去除+-,'ORtg','DRtg', 'BPM','PTS'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_7 缩小+-,'BPM','ORtg','DRtg', 
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_8 去除+-,'BPM','ORtg',通过模糊化降低'DRtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_9 去除+-,'BPM',通过模糊化降低'DRtg','ORtg'影响值
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg','ORtg']
    #v_10 去除+-,通过模糊化降低'DRtg','ORtg'影响值
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%','ORtg', 'DRtg', 'BPM']
    
    total_over_features = []
    for i in range(36):
        for feature in used_features:
            feature = feature + '_' + str(i)
            total_over_features.append(feature)
    print('选择的特征数量：',len(total_over_features))
    X = df[total_over_features].astype(float)
    Y = df['result'].astype(int)
    print('处理后数据：',len(X))
    
    if training:
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 
        return X_train[total_over_features], X_test[total_over_features], Y_train, Y_test
    else:
        return X[total_over_features], Y, total_over_features

def lgb_train(X_train, X_test, y_train, y_test):
    norm_features = []
    if  DRtg_Fuzzification:
        for i in range(36):
            for feat in ['DRtg']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
    if  ORtg_Fuzzification:
        for i in range(36):
            for feat in ['ORtg']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
    categorical_feature = norm_features
    model = lightgbm.LGBMClassifier(objective='binary', learning_rate=0.1, num_leaves = 30, min_data_in_leaf = 100, feature_fraction = 0.9, max_depth = 8, n_estimators=1500)
    callbacks = [log_evaluation(period=1), early_stopping(stopping_rounds=10)]
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='binary_logloss', callbacks = callbacks, categorical_feature = categorical_feature)
    joblib.dump(model, 'model/nba_mvp_lgb_v12.pkl')
    model = joblib.load('model/nba_mvp_lgb_v12.pkl')
    y_pred = model.predict(X_test, num_iteration=model.best_iteration_)


    print(np.average(y_pred))
    print('The acc of prediction is {}'.format(accuracy_score(y_test, y_pred)))
    print('The precision of prediction is {}'.format(precision_score(y_test, y_pred)))
    print('The recall of prediction is {}'.format(recall_score(y_test, y_pred)))

    X = pd.concat((X_train, X_test))
    y = np.append(y_train, y_test)
    y_all_pred = model.predict(X, num_iteration=model.best_iteration_)
    print('The acc of all prediction is {}'.format(accuracy_score(y, y_all_pred)))
    print('The precision of all prediction is {}'.format(precision_score(y, y_all_pred)))
    print('The recall of all prediction is {}'.format(recall_score(y, y_all_pred)))

def rawdata_load(extract_folder):
    file_path_list = []
    df_list = []
    # 遍历解压后的文件夹，并读取每个 CSV 文件
    for root, dirs, files in os.walk(extract_folder):
        for file in files:
            if file.endswith("basic_stats.csv"):
                file_path = os.path.join(root, file)
                file_path_list.append(file_path)
                # print(file_path)
                # 使用 pandas 读取 CSV 文件
                df_basic = pd.read_csv(file_path).drop_duplicates()
                df_advanced = pd.read_csv(file_path.replace('basic_stats.csv','advanced_stats.csv')).drop_duplicates()
                if df_basic['PTS'].dtype=='object':
                    df_basic = df_basic.drop(df_basic.index[-1])
                    df_advanced = df_advanced.drop(df_advanced.index[-1])
                # 连接两个文件：63162
                df = pd.concat([df_basic, df_advanced], axis=1)
                df = df.loc[:,~df.columns.duplicated()]
                # 将文件名作为新列添加到 DataFrame
                df['file_name'] = file_path.split('/')[-1]
                df_list.append(df)
    print(len(file_path_list),len(df_list)) 
    max_player = 18

    features = ['starters', 'team', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
        'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
        'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM',
        'file_name']
    # 连接所有球员数据
    new_features = []
    for i in range(max_player*2):
        for feat in features:
            new_feat = feat + '_' + str(i)
            new_features.append(new_feat)
    # print(len(features), len(new_features))

    result_features = ['score_ourside', 'score_opposite', 'result']

    new_data = pd.DataFrame(columns = new_features + result_features)
    for i in tqdm(range(len(file_path_list))):
        # for i in range(10):
        path = file_path_list[i]
        # print(path.split('/')[-1])
        df1 = df_list[i]
        filled_df1 = df1.reindex(range(max_player))
        #对手球队数据
        a = path.split('/')[-1].split('_')[3]
        b = path.split('/')[-1].split('_')[5]
        oppo_path = path.replace(a+'_vs_'+b, b+'_vs_'+a)
        # print(oppo_path,file_path_list.index(oppo_path))
        df2 = df_list[file_path_list.index(oppo_path)]
        filled_df2 = df2.reindex(range(max_player))
        # print(filled_df1["PTS"],filled_df2["PTS"])
        score_ourside = filled_df1["PTS"].astype(float).sum()
        score_opposite = filled_df2["PTS"].astype(float).sum()
        if score_ourside > score_opposite:
            result = 1
        else:
            result = 0
        
        feature_data1 = []
        feature_data2 = []
        for num in range(max_player):
            feature_data1 += filled_df1.loc[num, features].tolist() 
            feature_data2 += filled_df2.loc[num, features].tolist() 
            
        new_data.loc[i, new_features + result_features] = feature_data1 + feature_data2 + [score_ourside, score_opposite, result]
    return new_data

if __name__ == "__main__":
    data_path = './data/nba_mvp.csv'
    df1 = pd.read_csv(data_path)
    #加上24年数据
    folder_path = 'data/basketball_data_info/'
    df2 = rawdata_load(folder_path)
    df = pd.concat([df1, df2], axis=0)  
    print(len(df1),len(df2),len(df))
    data = data_load(df)
    X_train, X_test, y_train, y_test = data_prerpocess(data)
    lgb_train(X_train, X_test, y_train, y_test)

# 特征shap value值计算

In [9]:
# -*- coding: utf-8 -*-
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt
import multiprocessing
from tqdm import tqdm
import os
plt.rcParams['axes.unicode_minus']=False
#DRtg=5, ORtg不需要，效果可以
year = 2022
DRtg_bins = 8
PM_bins = 3
DRtg_Fuzzification = True
# DRtg_Fuzzification = False
PM_Fuzzification = True
# PM_Fuzzification = False

def data_load(data):
    df = data
    # print(df.info())
    # print(df.columns)
    # 将时间转换为浮点数
    time_features = []
    for i in range(36):
        feature = 'MP' + '_' + str(i)
        time_features.append(feature)
    # print(time_features) 
    
    df = df[~df['BPM_0'].astype(str).str.contains(':')]
    # print(df['BPM_0'].astype(str).str.contains(':'))
    print('清洗后数据：',len(df))
    df[time_features] = df[time_features].fillna('0:0')
    df[time_features] = df[time_features].map(lambda x: '0:0' if ':' not in str(x) else x)
    df[time_features] =df[time_features].map(lambda x: float(x.replace(':', '.')))
    df = df.fillna(0)
    if  DRtg_Fuzzification:
        #  #缩小'ORtg','DRtg'影响
        norm_features = []
        for i in range(36):
            for feat in ['DRtg']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
        # df = df.fillna(0)
        # # 转换为数值类型
        df[norm_features] = df[norm_features].apply(pd.to_numeric, errors='coerce')
        # 将负值变为0
        df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)
        # 将 norm_features 值聚合在一起
        norm_feature_values = df[norm_features].values.flatten().reshape(-1, 1)    
        print(norm_feature_values.shape)
        # 将数据划分为 10 个区间进行数值离散化处理
        kdb = KBinsDiscretizer(n_bins = DRtg_bins, encode = 'ordinal', strategy='quantile')
        kdb.fit(norm_feature_values)
        for norm_feature in norm_features:
            df[norm_feature] = kdb.transform(df[norm_feature].values.reshape(-1, 1))
    if  PM_Fuzzification:
        norm_features = []
        for i in range(36):
            for feat in ['+/-']:
                feature = feat + '_' + str(i)
                norm_features.append(feature)
        # # 转换为数值类型
        df[norm_features] = df[norm_features].apply(pd.to_numeric, errors='coerce')
        # 将负值变为0
        df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)
        # 将 norm_features 值聚合在一起
        norm_feature_values = df[norm_features].values.flatten().reshape(-1, 1)    
        print(norm_feature_values.shape)
        # 将数据划分为 10 个区间进行数值离散化处理
        kdb = KBinsDiscretizer(n_bins = PM_bins, encode = 'ordinal', strategy='quantile')
        kdb.fit(norm_feature_values)
        for norm_feature in norm_features:
            df[norm_feature] = kdb.transform(df[norm_feature].values.reshape(-1, 1))
    print(df[['DRtg_0','DRtg_35']])
    print(df[['+/-_0','+/-_35']])
    print(df.info())
    print(df.columns)
    return df

def data_prerpocess(df, training = True):
    
    #V_0：全有
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_1去除+-，DRtg
    used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'BPM']
    #V_2去除MP,+-，DRtg
    # used_features = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','BPM']
    #v_3 +-,'BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','DRtg']
    # V_4去除PTS,+-,'ORtg','BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_5 去除+-,'ORtg','DRtg', 'BPM'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_6 去除+-,'ORtg','DRtg', 'BPM','PTS'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_7 缩小+-,'BPM','ORtg','DRtg', 
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_8 去除+-,'BPM','ORtg',通过模糊化降低'DRtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_9 去除+-,'BPM',通过模糊化降低'DRtg','ORtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg','ORtg']
    #v_10 去除+-,通过模糊化降低'DRtg','ORtg'影响值
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%','ORtg', 'DRtg', 'BPM']
    #v_11 模糊化+-,'DRtg'
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    X_list = []
    Y_list = []
    total_over_features = []
    for i in range(36):
        for feature in used_features:
            feature = feature + '_' + str(i)
            total_over_features.append(feature)
    
    print('选择的特征数量：',len(total_over_features))
    X = df[total_over_features].astype(float)
    Y = df['result'].astype(int)
    X_list.append(X)
    Y_list.append(Y)
    # print('处理后数据：',len(X))
 
    # #打乱同一个队球员顺序添加训练数据进去
    # random.seed(0)
    # for sample in range(15):
    #     total_over_features = []
    #     for i in random.sample(range(18), 18):  # 打乱顺序
    #         for feature in used_features:
    #             new_feature = feature + '_' + str(i)
    #             total_over_features.append(new_feature)
                
    #     for i in random.sample(range(18,36), 18):  # 打乱顺序
    #         for feature in used_features:
    #             new_feature = feature + '_' + str(i)
    #             total_over_features.append(new_feature)
                
    #     print('特征顺序：',total_over_features)
    #     X1 = df[total_over_features].astype(float)
    #     Y1 = df['result'].astype(int)
    #     X_list.append(X1)
    #     Y_list.append(Y1)
        
    X = pd.concat(X_list, ignore_index=True)
    Y = pd.concat(Y_list, ignore_index=True)    
    print('处理后数据：',len(X))
    
    if training:
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 
        return X_train[total_over_features], X_test[total_over_features], Y_train, Y_test
    else:
        return X[total_over_features], Y, total_over_features

def compute_shap_values(shap_values_path):
    shap_values = np.load(shap_values_path)
    feat_num = len(shap_values[0]) // 36 #每个球员有33个特征
    mvp_values = []
    for i in tqdm(range(len(X))):
        shap_value = shap_values[i]
        mvp_value = 0
        for j in range(len(shap_value)):
            mvp_value += shap_value[j]
            if j % feat_num == feat_num - 1:
                mvp_values.append(mvp_value)
                mvp_value = 0
    abs_array = np.abs(np.array(mvp_values).reshape(-1,36))
    sum_first_three = np.sum(abs_array[:, :18], axis=1, keepdims=True)
    x = 0.5
    normalized_first_three = (abs_array[:, :18] / sum_first_three) * x

    # 后三列数值除以后三列的和并乘以系数（1-x）
    sum_last_three = np.sum(abs_array[:, 18:], axis=1, keepdims=True)
    normalized_last_three = (abs_array[:, 18:] / sum_last_three) * (1 - x)

    # 合并处理后的数组
    result_array = np.hstack((normalized_first_three, normalized_last_three))
    return result_array, np.array(mvp_values).reshape(-1,36), shap_values

def explain(X, model_path, shap_file_name):

    model = joblib.load(model_path)
    explainer = shap.TreeExplainer(model)
    print(explainer.expected_value)
    shap_values = explainer.shap_values(X)
    
    np.save(shap_file_name, shap_values)
    shap_values = np.load(shap_file_name)
    print(shap_values.shape)
# 队内排名
def rank(values, order = 1):
    if order == 1:
        sorted_indices = np.argsort(values)[::-1]
    else:
        sorted_indices = np.argsort(values)
    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(1, len(sorted_indices) + 1)

    return ranks, sorted_indices[0]

def process_data(args):
    df, feat_values, shap_values, id = args
    #V_0：全有
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_1去除+-，DRtg
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'BPM']
    #V_2去除MP,+-，DRtg
    # used_features = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','BPM']
    #V_3 +-,'ORtg','BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    # V_4去除PTS,+-,'ORtg','BPM'
    # used_features = ['MP','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_5 去除+-,'ORtg','DRtg', 'BPM'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_6 去除+-,'ORtg','DRtg', 'BPM','PTS'
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #      'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
    #v_7 缩小+-,'BPM','ORtg','DRtg', 
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    #v_8 去除+-,'BPM','ORtg',通过模糊化降低'DRtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg']
    #v_9 去除+-,'BPM',通过模糊化降低'DRtg','ORtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DRtg','ORtg']
    #v_10 去除+-,通过模糊化降低'DRtg','ORtg'影响值
    # used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
    #    'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
    #     'PTS', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
    #    'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%','ORtg', 'DRtg', 'BPM']
    #v_12 模糊化+-,'DRtg'
    used_features = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
        'PTS', '+/-','TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    
    feature_value = []
    id_feature = 'starters'
    id_features = [id_feature + '_' + str(i) for i in range(36)]
    

    score_features = ['score_ourside', 'score_opposite']
    total_over_features = []
    feat_num = len(used_features)
    
    for feature in used_features:
        feature_value.append(feature)
        feature_value.append(feature + '_shap_value')
    value_name = [feature_value[i] for i in range(1, len(feature_value), 2)]

    for i in range(36):
        for feature in used_features:
            feature = feature + '_' + str(i)
            total_over_features.append(feature)
  
    #ds, role_id
    new_data = pd.DataFrame(columns = ['file_name', 'result', 'starters', 'home_team', 'team_score'] + feature_value + [ 'shap_score', 'shap_order_team'])
    for i in tqdm(range(len(df))):
        result = df.loc[i, 'result']
        match_id = df.loc[i, 'file_name_0']
      
        self_shap_values = shap_values[i][:18]
        opp_shap_values = shap_values[i][18:]
        
        for j in range(18):
            #ds, role_id, feat_shap_value
            new_data.loc[36 * i + j, ['file_name', 'result', 'starters', 'team_score'] + used_features] = df.loc[i, ['file_name_0', 'result',  id_features[j], score_features[0]] + total_over_features[j * feat_num : (j + 1) * feat_num]].tolist()
            new_data.loc[36 * i + j, 'home_team'] = '1'
            new_data.loc[36 * i + j, value_name] = feat_values[i][j * feat_num : (j + 1) * feat_num] #n * feat_num * 6
            
        rows = [36 * i + n for n in range(18)]
        shap_ranks, shap_index1 = rank(self_shap_values)
        new_data.loc[rows, 'shap_order_team'] = shap_ranks
        
        
        for j in range(18, 36):
            new_data.loc[36 * i + j, ['file_name',  'starters', 'team_score'] + used_features] = df.loc[i, ['file_name_18', id_features[j], score_features[1]] + total_over_features[j * feat_num : (j + 1) * feat_num]].tolist()
            new_data.loc[36 * i + j, 'home_team'] = '0'
            new_data.loc[36 * i + j, 'result'] = 1 - df.loc[i, 'result']
            new_data.loc[36 * i + j, value_name] = feat_values[i][j * feat_num : (j + 1) * feat_num]
           
        rows = [36 * i + n for n in range(18,36)]
        shap_ranks, shap_index2 = rank(opp_shap_values, -1)
        new_data.loc[rows, 'shap_order_team'] = shap_ranks

    new_data['shap_score'] = shap_values.reshape(-1)
    return new_data
    
if __name__ == "__main__":
     
    data_path = './data/nba_mvp.csv'
    model_path = 'model/nba_mvp_lgb_v12.pkl'
    season = 'Regular_season/'
    # season = 'Finals/'
    df = pd.read_csv(data_path)
    data = data_load(df)
    
    if year == 2024:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2024_04_10|2024_04_11|2024_04_12|2024_04_13|2024_04_14'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2023:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2022:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2022_04_10'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]    
    if year == 2021:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_12|{}_01|{}_02|{}_03|{}_04|{}_05_0|2021_05_10|2021_05_11|2021_05_12|2021_05_13|2021_05_14|2021_05_15|2021_05_16'
                        .format(year-1,year,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2020:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04|{}_07|{}_08_0|2020_08_10|2020_08_11|2020_08_12|2020_08_13|2020_08_14'
                        .format(year-1,year-1,year-1,year,year,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2019:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2019_04_10'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2018:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2018_04_10|2018_04_11'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2017:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2017_04_10|2017_04_11|2017_04_12'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2016:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2016_04_10|2016_04_11|2016_04_12|2016_04_13'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2015:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2015_04_10|2015_04_11|2015_04_12|2015_04_13|2015_04_14|2015_04_15'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2014:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2014_04_10|2014_04_11|2014_04_12|2014_04_13|2014_04_14|2014_04_15|2014_04_16'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2013:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2013_04_10|2013_04_11|2013_04_12|2013_04_13|2013_04_14|2013_04_15|2013_04_16|2013_04_17'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2012:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|{}_04_1|2012_04_20|2012_04_21|2012_04_22|2012_04_23|2012_04_24|2012_04_25|2012_04_26'
                        .format(year-1,year-1,year-1,year,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2011:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2011_04_10|2011_04_11|2011_04_12|2011_04_13'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2010:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2010_04_10|2010_04_11|2010_04_12|2010_04_13|2010_04_14'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2009:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2009_04_10|2009_04_11|2009_04_12|2009_04_13|2009_04_14|2009_04_15'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2008:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2008_04_10|2008_04_11|2008_04_12|2008_04_13|2008_04_14|2008_04_15|2008_04_16'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2007:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|2007_04_10|2007_04_11|2007_04_12|2007_04_13|2007_04_14|2007_04_15|2007_04_16|2007_04_17|2007_04_18'
                        .format(year-1,year-1,year-1,year,year,year,year), regex=True)).any(axis=1)]
    if year == 2006:
        data = data[data.apply(lambda x: x.astype(str).str.contains('{}_10|{}_11|{}_12|{}_01|{}_02|{}_03|{}_04_0|{}_04_1'
                        .format(year-1,year-1,year-1,year,year,year,year,year), regex=True)).any(axis=1)]
        
 
    print('{}比赛数据：'.format(year),len(data))
    X, Y, total_over_features = data_prerpocess(data, training= False) 
    shap_file_name = './shap/'+season+'{}_nba_mvp_shap_values_v12.npy'.format(year)
    # if not os.path.exists(shap_file_name):
    explain(X, model_path, shap_file_name)
    money_values, shap_values, feat_values = compute_shap_values(shap_file_name)

    # 分割数据
    num_cores = 10
    chunk_size = len(X) // num_cores
    chunks = [(data[i:i + chunk_size].reset_index(drop = True), feat_values[i : i + chunk_size], shap_values[i : i + chunk_size], i // chunk_size * 100 // num_cores) for i in range(0, len(data), chunk_size)]
    # 使用线程池处理数据

    # 创建进程池，并利用多个核心进行并行处理
    with multiprocessing.Pool(processes=num_cores) as pool:
        results = pool.map(process_data, chunks)

    # 合并处理结果
    data_list = [results[i] for i in range(num_cores)]
    
    processed_data = pd.concat(data_list).reset_index(drop = True)
    processed_data.to_csv('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))


/tmp/ipykernel_2397963/967433280.py:357: DtypeWarning: Columns (37,74,111,148,185,222,259,296,333,370,407,444,481,495,496,497,532,533,534,535,570,571,572,573,608,609,610,611,646,647,648,649,684,721,758,795,832,869,906,943,980,1017,1054,1091,1128,1165,1179,1180,1181,1216,1217,1218,1219,1254,1255,1256,1257,1292,1293,1294,1295,1330,1331,1332,1333,1368) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


清洗后数据： 63112


/tmp/ipykernel_2397963/967433280.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[time_features] = df[time_features].fillna('0:0')
/tmp/ipykernel_2397963/967433280.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[time_features] = df[time_features].map(lambda x: '0:0' if ':' not in str(x) else x)
/tmp/ipykernel_2397963/967433280.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

(2272032, 1)


/root/miniconda3/envs/mvp_shap/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/tmp/ipykernel_2397963/967433280.py:68: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[norm_features] = df[norm_features].applymap(lambda x: 0 if x < 0 else x)


...

/root/miniconda3/envs/mvp_shap/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


2022比赛数据： 2460
选择的特征数量： 1260
处理后数据： 2460
None


/root/miniconda3/envs/mvp_shap/lib/python3.9/site-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


(2460, 1260)


100%|██████████| 246/246 [02:20<00:00,  1.76it/s]


# 特征分析图

In [10]:
# 绘制summary分析图并保存
def summary_bar_plot(shap_values, max_display=20, show=False, save_path=None):
    plt.style.use('ggplot')
    shap.plots.bar(shap_values, max_display=max_display, show=show)
    if save_path:
        plt.savefig(save_path.encode(encoding='utf-8'), dpi=200, bbox_inches='tight')
    plt.close('all')

def summary_dot_plot(shap_values, max_display=20, show=False, save_path=None):
    plt.style.use('ggplot')
    shap.plots.beeswarm(shap_values, max_display=max_display, show=show)
    if save_path:
        plt.savefig(save_path.encode(encoding='utf-8'), dpi=200, bbox_inches='tight')
    plt.close('all')

# 绘制summary分析图并保存
save_path = 'shap/'+season+'{}_nba_mvp_summary_dot_v12.png'.format(year)
summary_dot_plot(shap.Explanation(
values=feat_values, 
# base_values = 0,
data=np.array(X),
feature_names=total_over_features), max_display=50, show=False, save_path=save_path)

save_path = 'shap/'+season+'{}_nba_mvp_summary_bar_v12.png'.format(year)
summary_bar_plot(shap.Explanation(
values=feat_values, 
# base_values = np.array(0),
data=np.array(X),
feature_names=total_over_features), max_display=50, show=False, save_path=save_path)


# mvp评估

In [1]:
import numpy as np
from sklearn.metrics import ndcg_score, average_precision_score
# MRR (Mean Reciprocal Rank)
def mean_reciprocal_rank(ranking):
    for i, rank in enumerate(ranking):
        if rank == 1:
            return 1 / (i + 1)
    return 0
season = 'Regular_season/'
year = 2024
#2024
if year == 2024:
    mvp_voting = ['Nikola Jokić', 'Shai Gilgeous-Alexander', 'Luka Dončić','Giannis Antetokounmpo',
                  'Jalen Brunson','Jayson Tatum', 'Anthony Edwards', 'Domantas Sabonis', 
                  'Kevin Durant']
#2023
if year == 2023:
    mvp_voting = ['Joel Embiid', 'Nikola Jokić', 'Giannis Antetokounmpo', 'Jayson Tatum', 
                  'Shai Gilgeous-Alexander', 'Donovan Mitchell', 'Domantas Sabonis', 'Luka Dončić'
                  , 'Stephen Curry', 'Jimmy Butler', "De'Aaron Fox", 'Jalen Brunson', 'Ja Morant']
#2022
if year == 2022:
    mvp_voting = ['Nikola Jokić', 'Joel Embiid', 'Giannis Antetokounmpo', 'Devin Booker',
                  'Luka Dončić', 'Jayson Tatum', 'Ja Morant', 'Stephen Curry', 'Chris Paul',
                  'DeMar DeRozan', 'Kevin Durant', 'LeBron James']
#2021
if year == 2021:
    mvp_voting = ['Nikola Jokić', 'Joel Embiid', 'Stephen Curry', 'Giannis Antetokounmpo',
                  'Chris Paul', 'Luka Dončić', 'Damian Lillard', 'Julius Randle', 'Derrick Rose',
                  'Rudy Gobert', 'Russell Westbrook', 'Ben Simmons', 'James Harden','LeBron James', 'Kawhi Leonard']
#2020
if year == 2020:
    mvp_voting = ['Giannis Antetokounmpo', 'LeBron James', 'James Harden', 'Luka Dončić',
    'Kawhi Leonard', 'Anthony Davis', 'Chris Paul','Damian Lillard','Nikola Jokić',
    'Pascal Siakam','Jimmy Butler','Jayson Tatum']
#2019
if year == 2019:
    mvp_voting = ['Giannis Antetokounmpo', 'James Harden', 'Paul George', 'Nikola Jokić',
    'Stephen Curry','Damian Lillard','Joel Embiid','Kevin Durant','Kawhi Leonard','Russell Westbrook',
    'Rudy Gobert','LeBron James']
#2018
if year == 2018:
    mvp_voting = ['James Harden', 'LeBron James', 'Anthony Davis', 'Damian Lillard',              
    'Russell Westbrook','Giannis Antetokounmpo','Kevin Durant','DeMar DeRozan','LaMarcus Aldridge',
    'Jimmy Butler','Stephen Curry','Joel Embiid','Victor Oladipo']
#2017
if year == 2017:
    mvp_voting = ['Russell Westbrook', 'James Harden', 'Kawhi Leonard', 'LeBron James',
    'Isaiah Thomas','Stephen Curry','Giannis Antetokounmpo','John Wall','Anthony Davis',
    'Kevin Durant','DeMar DeRozan']
#2016
if year == 2016:
    mvp_voting = ['Stephen Curry', 'Kawhi Leonard', 'LeBron James', 'Russell Westbrook',
    'Kevin Durant','Chris Paul','Draymond Green','Damian Lillard','James Harden',
    'Kyle Lowry']
#2020
if year == 2015:
    mvp_voting = ['Stephen Curry', 'James Harden', 'LeBron James', 'Russell Westbrook',
    'Anthony Davis','Chris Paul','LaMarcus Aldridge','Marc Gasol','Blake Griffin',
    'Tim Duncan','Kawhi Leonard','Klay Thompson']
#2020
if year == 2014:
    mvp_voting = ['Kevin Durant', 'LeBron James', 'Blake Griffin', 'Joakim Noah',
    'James Harden','Stephen Curry','Chris Paul','Al Jefferson','Paul George','LaMarcus Aldridge',
    'Kevin Love','Tim Duncan','Tony Parker','Dirk Nowitzki','Carmelo Anthony',
    'Goran Dragić','Mike Conley']
#2020
if year == 2013:
    mvp_voting = ['LeBron James', 'Kevin Durant', 'Carmelo Anthony', 'Chris Paul',
    'Kobe Bryant','Tony Parker','Tim Duncan','James Harden','Russell Westbrook','Dwyane Wade',
    'Stephen Curry','Kevin Garnett','Marc Gasol','Ty Lawson','David Lee','Joakim Noah']
#2020
if year == 2012:
    mvp_voting = ['LeBron James', 'Kevin Durant', 'Chris Paul', 'Kobe Bryant',
    'Tony Parker','Kevin Love','Dwight Howard','Rajon Rondo','Steve Nash','Dwyane Wade',
    'Derrick Rose','Dirk Nowitzki','Russell Westbrook','Tim Duncan','Joe Johnson']  
#2020
if year == 2011:
    mvp_voting = ['Derrick Rose', 'Dwight Howard', 'LeBron James', 'Kobe Bryant',
    'Kevin Durant','Dirk Nowitzki','Dwyane Wade','Manu Ginóbili',"Amar'e Stoudemire",
    'Blake Griffin','Rajon Rondo','Tony Parker','Chris Paul']
#2020
if year == 2010:
    mvp_voting = ['LeBron James', 'Kevin Durant', 'Kobe Bryant', 'Dwight Howard','Dwyane Wade',
    'Carmelo Anthony','Dirk Nowitzki','Steve Nash','Deron Williams',"Amar'e Stoudemire",
    'Manu Ginóbili','Chauncey Billups','Chris Bosh','Stephen Jackson','Joe Johnson']
#2020
if year == 2009:
    mvp_voting = ['LeBron James', 'Kobe Bryant', 'Dwyane Wade', 'Dwight Howard', 'Chris Paul',               
    'Chauncey Billups','Paul Pierce','Tony Parker','Brandon Roy','Dirk Nowitzki',
    'Tim Duncan','Yao Ming']
#2020
if year == 2008:
    mvp_voting = ['Kobe Bryant', 'Chris Paul', 'Kevin Garnett', 'LeBron James','Dwight Howard',
    "Amar'e Stoudemire", 'Tim Duncan','Tracy McGrady','Steve Nash','Manu Ginóbili','Dirk Nowitzki',
    'Deron Williams','Carmelo Anthony','Carlos Boozer','Antawn Jamison','Paul Pierce','Rasheed Wallace']
#2020
if year == 2007:
    mvp_voting = ['Dirk Nowitzki', 'Steve Nash', 'Kobe Bryant', 'Tim Duncan','LeBron James',
    'Tracy McGrady','Chris Bosh','Gilbert Arenas','Carlos Boozer','Kevin Garnet','Chauncey Billups',
    "Shaquille O'Neal",'Dwyane Wade',"Amar'e Stoudemire",'Carmelo Anthony','Baron Davis','Tony Parker']
#2020
if year == 2006:
    mvp_voting = ['Steve Nash', 'LeBron James', 'Dirk Nowitzki', 'Kobe Bryant', 'Chauncey Billups',
    'Dwyane Wade','Elton Brand','Tim Duncan','Tony Parker','Allen Iverson','Shawn Marion']
                      


In [6]:
#球员在所有胜场的shapley值排名平均
topk = 3
topk = len(mvp_voting)
import pandas as pd
from scipy.stats import spearmanr
data_value = pd.read_csv('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))
data_value= data_value[data_value['MP'] != 0.0]
# print(data_value['MP'])
print('上场人次：',len(data_value))
# print('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))
#最低场次限制 
# {24:130, 23:110, 22:109, 21:87, 20:113, 19:109, 18:100, 17:122
# 16:130, 15:127, 14:123, 13:130, 12:77, 11:130, 10:130, 09:130
# 08:130, 07:79, 06:130}
result_df = \
data_value[data_value['starters'].
           map(data_value['starters'].value_counts()) > 130]
#胜场
result_df_win = result_df[result_df['result'] == 1]
mvp_by_shap = result_df_win.groupby('starters')['shap_order_team'].mean().sort_values()
print(len(mvp_by_shap),mvp_by_shap[:10])
# mvp = mvp_by_shap[mvp_by_shap == mvp_by_shap.min()]
# print('mvp:',mvp) # Nikola Jokić 尼古拉·约基奇:1 ,Joel Embiid 乔尔·恩比德:7
mvp_rank = mvp_by_shap.rank().sort_values()
# print('Devin Booker:', mvp_rank['Devin Booker'] ,mvp_by_shap['Devin Booker'])

rank_mean_err = 0
predicted_labels = []
for i in range(topk):
    predicted_labels.append(mvp_rank[mvp_voting[i]]-1)
    rank_mean_err += abs(mvp_rank[mvp_voting[i]]-i-1)
print('rank_mean_err:', rank_mean_err/topk)
actual_labels = list(range(1, 1+topk))
print(actual_labels)
print(predicted_labels)
# 计算Spearman相关系数
correlation, p_value = spearmanr(actual_labels, predicted_labels)
print("Spearman相关系数:", correlation)
# print("p值:", p_value)
#前13名计算召回率=精度：
count = sum(1 for i in predicted_labels if i < topk)
print("Precision or recall:", count/topk)


上场人次： 52788
180 starters
Victor Wembanyama        2.210526
Nikola Jokić             2.715596
Daniel Gafford           2.933333
Rudy Gobert              3.245283
Nick Richards            3.357143
Nic Claxton              3.518519
Giannis Antetokounmpo    3.888889
Anthony Davis            3.913043
Bam Adebayo              4.825000
Zion Williamson          4.857143
Name: shap_order_team, dtype: float64
rank_mean_err: 26.22222222222222
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1.0, 15.0, 18.0, 6.0, 67.0, 61.0, 29.0, 17.0, 58.0]
Spearman相关系数: 0.5833333333333334
Precision or recall: 0.2222222222222222


In [7]:
#球员在所有场的shapley值排名平均
# data_value = pd.read_csv('./data/{}_nba_mvp_values_v12.csv'.format(year))
# result_df = data_value[data_value['starters'].map(data_value['starters'].value_counts()) > 130]

mvp_by_shap = result_df.groupby('starters')['shap_order_team'].mean().sort_values()
print(len(mvp_by_shap),mvp_by_shap[:10])
# mvp = mvp_by_shap[mvp_by_shap == mvp_by_shap.min()]
# print('mvp:',mvp) # Nikola Jokić 尼古拉·约基奇:1 ,Joel Embiid 乔尔·恩比德:5

mvp_rank = mvp_by_shap.rank().sort_values()
rank_mean_err = 0
predicted_labels = []
for i in range(topk):
    predicted_labels.append(mvp_rank[mvp_voting[i]]-1)
    rank_mean_err += abs(mvp_rank[mvp_voting[i]]-i-1)
print('rank_mean_err:', rank_mean_err/topk)
actual_labels = list(range(1, 1+topk))
print(actual_labels)
print(predicted_labels)
# 计算Spearman相关系数
correlation, p_value = spearmanr(actual_labels, predicted_labels)
print("Spearman相关系数:", correlation)
# print("p值:", p_value)
#前13名计算召回率=精度：
count = sum(1 for i in predicted_labels if i < topk)
print("Precision or recall:", count/topk)

180 starters
Nikola Jokić               4.859873
Victor Wembanyama          5.239437
Rudy Gobert                5.730263
Giannis Antetokounmpo      6.369863
Jusuf Nurkić               6.506579
Shai Gilgeous-Alexander    6.933333
Anthony Davis              6.948052
Chet Holmgren              7.042683
Andre Drummond             7.050633
Daniel Gafford             7.054054
Name: shap_order_team, dtype: float64
rank_mean_err: 23.88888888888889
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[0.0, 5.0, 14.0, 3.0, 80.0, 39.0, 31.0, 17.0, 62.0]
Spearman相关系数: 0.7166666666666667
Precision or recall: 0.3333333333333333


In [8]:
#球员在所有场的shapley值平均
# data_value = pd.read_csv('./data/{}_nba_mvp_values_v12.csv'.format(year))
# result_df = data_value[data_value['starters'].map(data_value['starters'].value_counts()) > 130]
result_df1 = result_df[result_df['home_team'] == 1]
result_df2 = result_df[result_df['home_team'] == 0]

# print(result_df['starters'].value_counts())
mvp_by_shap1 = result_df1.groupby('starters')['shap_score'].mean()
mvp_by_shap2 = result_df2.groupby('starters')['shap_score'].mean()
# print(len(mvp_by_shap1),mvp_by_shap1[:10])
# print(len(mvp_by_shap2),mvp_by_shap2[:10])
mvp_by_shap = (mvp_by_shap1 - mvp_by_shap2).sort_values(ascending=False)
print(mvp_by_shap[:10])

mvp_rank = mvp_by_shap.rank(ascending=False)
# print('Devin Booker:', 'rank:', mvp_rank['Devin Booker'] , 'mvp Shapley:', mvp_by_shap['Devin Booker'])

rank_mean_err = 0
predicted_labels = []
for i in range(topk):
    predicted_labels.append(mvp_rank[mvp_voting[i]]-1)
    rank_mean_err += abs(mvp_rank[mvp_voting[i]]-i-1)
print('rank_mean_err:', rank_mean_err/topk)
actual_labels = list(range(1, 1+topk))
print(actual_labels)
print(predicted_labels)
# 计算Spearman相关系数
correlation, p_value = spearmanr(actual_labels, predicted_labels)
print("Spearman相关系数:", correlation)
# print("p值:", p_value)
#前13名计算召回率=精度：
count = sum(1 for i in predicted_labels if i < topk)
print("Precision or recall:", count/topk)

starters
Nikola Jokić               1.311513
Victor Wembanyama          0.977095
Rudy Gobert                0.962410
Shai Gilgeous-Alexander    0.831670
Anthony Davis              0.604502
Giannis Antetokounmpo      0.511388
Chet Holmgren              0.481327
Bam Adebayo                0.404070
Jusuf Nurkić               0.388700
Isaiah Hartenstein         0.386903
Name: shap_score, dtype: float64
rank_mean_err: 20.666666666666668
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[0.0, 3.0, 12.0, 5.0, 73.0, 23.0, 20.0, 22.0, 64.0]
Spearman相关系数: 0.7666666666666667
Precision or recall: 0.3333333333333333


In [9]:
#输球负贡献置零
# result_df = pd.read_csv('./data/'+season+'{}_nba_mvp_values_v12.csv'.format(year))
result_df1 = result_df[result_df['home_team'] == 1]
result_df2 = result_df[result_df['home_team'] == 0]
#负场
result_df_loss1 = result_df1[result_df1['result'] == 0]
result_df_loss1['shap_score'][result_df_loss1['shap_score'] < 0] = 0
result_df_loss2 = result_df2[result_df2['result'] == 0]
result_df_loss2['shap_score'][result_df_loss2['shap_score'] > 0] = 0
result_df1 = pd.concat([result_df_loss1, result_df1[result_df1['result'] == 1]])
result_df2 = pd.concat([result_df_loss2, result_df2[result_df2['result'] == 1]])
# print(result_df['starters'].value_counts())
mvp_by_shap1 = result_df1.groupby('starters')['shap_score'].mean()
mvp_by_shap2 = result_df2.groupby('starters')['shap_score'].mean()

mvp_by_shap = (mvp_by_shap1 - mvp_by_shap2).sort_values(ascending=False)
print(mvp_by_shap[:10])

mvp_rank = mvp_by_shap.rank(ascending=False)
# print('Devin Booker:', 'rank:', mvp_rank['Devin Booker'] , 'mvp Shapley:', mvp_by_shap['Devin Booker'])

rank_mean_err = 0
predicted_labels = []
for i in range(topk):
    predicted_labels.append(mvp_rank[mvp_voting[i]]-1)
    rank_mean_err += abs(mvp_rank[mvp_voting[i]]-i-1)
print('rank_mean_err:', rank_mean_err/topk)
actual_labels = list(range(1, 1+topk))
print(actual_labels)
print(predicted_labels)
# 计算Spearman相关系数
correlation, p_value = spearmanr(actual_labels, predicted_labels)
print("Spearman相关系数:", correlation)
# print("p值:", p_value)
#前13名计算召回率=精度：
count = sum(1 for i in predicted_labels if i < topk)
print("Precision or recall:", count/topk)

starters
Nikola Jokić               1.635002
Victor Wembanyama          1.571314
Rudy Gobert                1.461918
Anthony Davis              1.329768
Shai Gilgeous-Alexander    1.316598
Bam Adebayo                1.154977
Giannis Antetokounmpo      1.044392
Daniel Gafford             1.029346
Chet Holmgren              0.990396
Isaiah Hartenstein         0.866604
Name: shap_score, dtype: float64
rank_mean_err: 16.555555555555557
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[0.0, 4.0, 12.0, 6.0, 56.0, 29.0, 16.0, 14.0, 48.0]
Spearman相关系数: 0.75
Precision or recall: 0.3333333333333333


/tmp/ipykernel_2412350/1423034049.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  result_df_loss1['shap_score'][result_df_loss1['shap_score'] < 0] = 0
/tmp/ipykernel_2412350/1423034049.py:7: SettingWithCopyWarning: 
A value is trying to 